<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> Oversampling mixed data-set with MGS-GRF </h1></center>

This notebook illustrates the way of uysing our propposed method with an example.

In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pathlib import Path
from collections import Counter

In [2]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

from mgs_grf import MGSGRFOverSampler

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> BankChurners data </h1></center>

## Import data and split


In [3]:
## Import of the original data
from experiments.data.data import load_BankChurners_data_

X, y = load_BankChurners_data_()
numeric_features = [0,2,7,8,9,10,11,12,13,14,15,16,17,18]
X = X[:,numeric_features]

In [4]:
## Splitting the data intro train and test set
X_train_imbalanced, X_test, y_train_imbalanced, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("Bankchurners 1% imbalance ratio : ", Counter(y_train_imbalanced))

Bankchurners 1% imbalance ratio :  Counter({0: 6802, 1: 66})


In [5]:
X_train_imbalanced

array([[59, 1, 51, ..., 103, 0.717, 0.071],
       [48, 2, 27, ..., 80, 0.86, 0.339],
       [57, 1, 41, ..., 75, 0.562, 0.696],
       ...,
       [44, 4, 34, ..., 68, 0.619, 0.632],
       [50, 3, 38, ..., 41, 0.64, 0.187],
       [59, 1, 53, ..., 33, 0.32, 0.346]], dtype=object)

## Apply MGS-GRF and compare to applying no rebalacing strategies

In [6]:
## Apply MGS-GRF procedure to oversample the data
mgs_grf = MGSGRFOverSampler(random_state=0)
X_train_balanced, y_train_balanced = mgs_grf.fit_resample(X_train_imbalanced, y_train_imbalanced)
print("Augmented data : ", Counter(y_train_balanced))

Augmented data :  Counter({0: 6802, 1: 6802})


/home/abdoulaye_sakho/S2/mgs-grf/mgs_grf/over_sampling.py:372: UserWarning: MGSGRFOverSampler called with continuous features only.
  warnings.warn('MGSGRFOverSampler called with continuous features only.')


In [7]:
# Fit the final classifier on the augmented data
clf = lgb.LGBMClassifier(n_estimators=100, verbosity=-1, random_state=0)
clf.fit(X_train_balanced, y_train_balanced)

## Evaluation on test set
y_pred_probas_mgs_grf = clf.predict_proba(X_test)[:,1]
print("PR AUC induced by MGS-GRF : ", average_precision_score(y_test, y_pred_probas_mgs_grf))

PR AUC induced by MGS-GRF :  0.5993393573877037


/home/abdoulaye_sakho/miniconda3/envs/dev_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/abdoulaye_sakho/miniconda3/envs/dev_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [8]:
## Comparison when applying no rebalancing strategy
clf_imbalanced = lgb.LGBMClassifier(n_estimators=100, verbosity=-1, random_state=0)
clf_imbalanced.fit(X_train_imbalanced, y_train_imbalanced)

y_pred_probas_imbalanced = clf_imbalanced.predict_proba(X_test)[:,1]
print("PR AUC induced by applying no rebalancing strategy : ", average_precision_score(y_test, y_pred_probas_imbalanced))

/home/abdoulaye_sakho/miniconda3/envs/dev_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


PR AUC induced by applying no rebalancing strategy :  0.44497912661955746


/home/abdoulaye_sakho/miniconda3/envs/dev_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> END </h1></center>